In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns

In [2]:
weekly = load_data("Weekly")

In [3]:
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [7]:
y = weekly["Direction"].transform(lambda x : x == "Up")

In [8]:
design = MS(['Lag1', 'Lag2'])
X = design.fit_transform(raw_X)

In [9]:
model = sm.Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4


In [51]:
%%capture
mistakes = []
for i in range(len(X)):
    X_oof = X.iloc[i];
    y_oof = y.iloc[i];
    
    X_fold = X.drop(i);
    y_fold = y.drop(i);
    
    X_fold = design.fit_transform(X_fold);
    model = sm.Logit(y_fold, X_fold).fit();

    mistakes.append((model.predict(X_oof) > 0.5).item() != y_oof)

In [61]:
np.mean(mistakes)
# 45% error rates, is high but in predicting the stock market it looks pretty good

0.44995408631772266

In [62]:
# let's check the null classifier 
null_mistakes = [True != val for val in y]
np.mean(null_mistakes)

#lower than the logistic regression, it looks like our model is terrible at predicting


0.4444444444444444